In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Apr 20 06:44:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 16. HuggingFace 커스텀 프로젝트 만들기

### 16-1. 들어가며

Huggingface transformers framework를 활용하여 빠르게 자신만의 커스텀 프로젝트를 구성

### 16-2. GLUE dataset과 Huggingface

****GLUE Benchmark Dataset****

NLP 모델의 성능을 측정하기 위한 데이터셋 [General Language Understanding Evaluation(GLUE) benchmark Dataset](https://gluebenchmark.com/)  

CoLA : 문법에 맞는 문장인지 판단  
MNLI : 두 문장의 관계 판단(entailment, contradiction, neutral)  
MNLI-MM : 두 문장이 안 맞는지 판단  
MRPC : 두 문장의 유사도 평가  
SST-2 : 감정분석  
STS-B : 두 문장의 유사도 평가  
QQP : 두 질문의 유사도 평가  
QNLI : 질문과 paragraph 내 한 문장이 함의 관계(entailment)인지 판단  
RTE : 두 문장의 관계 판단(entailment, not_entailment)  
WNLI : 원문장과 대명사로 치환한 문장 사이의 함의 관계 판단  

GLUE 홈페이지에는 위 [10가지 task에 대한 상세한 설명]  (https://gluebenchmark.com/tasks), 그리고 [Leaderboard]  (https://gluebenchmark.com/leaderboard)를 운영

아래 코드는 10가지 GLUE task 중 'mrpc' task를 수행하는 예제 코드

 cd ~/aiffel && git clone https://github.com/huggingface/transformers.git  
 cd transformers && pip install -e .  
 pip install datasets  
 pip install huggingface-hub==0.1.0  

 cd ~/aiffel/transformers  
 python examples/tensorflow/text-classification/run_glue.py \
	--model_name_or_path bert-base-cased \
	--task_name mrpc \
	--output_dir ./models/mrpc \
	--overwrite_output_dir \
	--do_train \
	--do_eval \
	--num_train_epochs 1 \
	--save_steps 20000

### ****16-3. 커스텀 프로젝트 제작 (1) Processor****

****mrpc 데이터셋 분석****

In [ ]:
import os
import numpy as np
from argparse import ArgumentParser
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoConfig
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures


In [ ]:
data, info = tfds.load('glue/mrpc', with_info=True)
info.splits['train'].num_examples

In [ ]:
data['train'].take(1)

In [ ]:
examples = data['train'].take(1)
for example in examples:
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']
    label = example['label']
    print(sentence1)
    print(sentence2)
    print(label)

****Processor의 활용****

아래는 추상클래스인 `Processor`를 한번 상속받은 후, Sequence Classification task를 수행하는 모델의 `Processor` 추상클래스인 `DataProcessor`

In [ ]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))


In [ ]:
class MrpcProcessor(DataProcessor):
    """Processor for the MRPC data set (GLUE version)."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["sentence1"].numpy().decode("utf-8"),
            tensor_dict["sentence2"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            text_b = line[4]
            label = None if set_type == "test" else line[0]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples


In [ ]:
processor = MrpcProcessor()
examples = data['train'].take(1)

for example in examples:
    print('------원본데이터------')
    print(example)  
    example = processor.get_example_from_tensor_dict(example)
    print('------processor 가공데이터------')
    print(example)

In [ ]:
examples = (data['train'].take(1))
for example in examples:
    example = processor.get_example_from_tensor_dict(example)
    example = processor.tfds_map(example)
    print(example)

#InputExample(guid=1680, text_a='The identical rovers will act as robotic geologists , searching for evidence of past water .', text_b='The rovers act as robotic geologists , moving on six wheels .', label='0')

In [ ]:
label_list = processor.get_labels()
label_list

In [ ]:
label_map = {label: i for i, label in enumerate(label_list)}
label_map

### ****16-4. 커스텀 프로젝트 제작 (2) Tokenizer와 Model****

`MRPCProcessor` 클래스와 framework를 결합시켜 나가는 과정을 진행

`tokenizer`와 `model`을 생성

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



In [1]:
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid))
        print("features: %s" % features[i])

    return features

In [2]:
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )

`_glue_convert_examples_to_features()` 함수는 processor가 생성한 example을 `tokenizer`로 인코딩하여 feature로 변환

`tf_glue_convert_examples_to_features()` 함수는 내부적으로 `_glue_convert_examples_to_features()`를 호출해서 얻은 feature를 바탕으로 `tf.data.Dataset`을 생성하여 리턴

In [ ]:
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)

tf_glue_convert_examples_to_features() 함수가 최종적으로 모델에 전달될 tf.data.Dataset 인스턴스를 생성

In [ ]:
examples = train_dataset.take(1)
for example in examples:
    print(example)

In [ ]:
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)
train_dataset_batch = train_dataset.shuffle(100).batch(16).repeat(2)

In [ ]:
# validation 데이터셋
validation_dataset = tf_glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, processor=processor)
validation_dataset_batch = validation_dataset.shuffle(100).batch(16)

In [ ]:
# test 데이터셋
test_dataset = tf_glue_convert_examples_to_features(data['test'], tokenizer, max_length=128, processor=processor)
test_dataset_batch = test_dataset.shuffle(100).batch(16)

### ****16-5. 커스텀 프로젝트 제작 (3) Train/Evaluation과 Test****

****tf.keras.model 을 활용한 학습****

In [ ]:
num_classes = len(processor.get_labels())
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

model.summary()

In [ ]:
# 이전 스텝에서 배치처리를 진행한 데이터셋(xxxx_dataset_batch)을 활용
model.fit(train_dataset_batch, epochs=2, steps_per_epoch=115, 
                validation_data=validation_dataset_batch)

In [ ]:
result = model.evaluate(test_dataset_batch)
print(result)

In [ ]:
import os
# output_dir = os.getenv('HOME')+'/aiffel/transformers'
# output_eval_file = os.path.join(output_dir, "eval_results.txt")

output_eval_file =  "eval_results.txt"

with open(output_eval_file, "w") as writer:
    for i, v in enumerate(result) :
        if i == 0 :
            writer.write("Loss = %f\t" %(v))
        if i == 1 :
            writer.write("Accuracy = %f\n" %(v))
print("완료=3")

#파일에 쓴 테스트 결과 확인
# !cat ~/aiffel/transformers/eval_results.txt
!cat eval_results.txt

#완료=3
#Loss = 0.580097	Accuracy = 0.690435

****TFTrainer를 활용한 학습****

Huggingface의 `TFTrainer`를 활용해 학습을 진행

`TFTrainer`를 활용하기 위해서는 `TFTrainingArguments`에 학습 관련 설정을 미리 지정해 두어야 함.

In [ ]:
# TFTrainer을 활용하는 형태로 모델 재생성
from transformers import (
    AutoConfig,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    PreTrainedTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
    glue_compute_metrics,
    glue_convert_examples_to_features,
    glue_output_modes,
    glue_processors,
    glue_tasks_num_labels,
)

# output_dir = os.getenv('HOME')+'/aiffel/transformers'
output_dir = '.'

training_args = TFTrainingArguments(
    output_dir=output_dir,            # output이 저장될 경로
    num_train_epochs=3,              # train 시킬 총 epochs
    per_device_train_batch_size=16,  # 각 device 당 batch size
    per_device_eval_batch_size=64,   # evaluation 시에 batch size
    warmup_steps=500,                # learning rate scheduler에 따른 warmup_step 설정
    weight_decay=0.01,                 # weight decay
    logging_dir='./logs',                 # log가 저장될 경로
    do_train=True,
    do_eval=True,
    eval_steps=1000
)

max_seq_length=128
task_name = "mrpc"

with training_args.strategy.scope():
    model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

아래에서 생성하게 될 TFTrainer의 인자로 넘겨주어야 할 것 중에 compute_metrics 메소드가 있습니다. 이것은 task가 classification인지 regression인지에 따라 모델의 출력 형태가 달라지므로 task별로 적합한 출력 형식을 고려해 모델의 성능을 계산하는 방법을 미리 지정

In [ ]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    if output_mode == "classification":
        preds = np.argmax(p.predictions, axis=1)
    elif output_mode == "regression":
        preds = np.squeeze(p.predictions)
    return glue_compute_metrics(task_name, preds, p.label_ids)

output_mode = glue_output_modes[task_name]
output_mode

TFTrainer를 활용할 때 데이터셋에서 잊지 않아야 할 것이 tf.data.experimental.assert_cardinality()를 데이터셋에 적용해 주는 것입니다. 이를 호출해 주지 않으면 TFTrainer.train()에서 assert fail이 발생

In [ ]:
train_dataset = train_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['train'].num_examples))
validation_dataset = validation_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['validation'].num_examples))
test_dataset = test_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['test'].num_examples))

TFTrainer를 생성

In [ ]:
trainer = TFTrainer(
    model=model,                           # 학습시킬 model
    args=training_args,                  # TFTrainingArguments을 통해 설정한 arguments
    train_dataset=train_dataset,    # training dataset
    eval_dataset=validation_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)

In [ ]:
# Training
if training_args.do_train:
    trainer.train()
    trainer.save_model()
    tokenizer.save_pretrained(training_args.output_dir)

학습이 끝나면 Evaluation을 진행하여 위 model.fit()으로 학습한 경우와 비교해 봅시다.

In [ ]:
import numpy as np

# Evaluation
results = {}
if training_args.do_eval:
    result = trainer.evaluate()
    output_eval_file = os.path.join(training_args.output_dir, "eval_results2.txt")

    with open(output_eval_file, "w") as writer:
        for key, value in result.items():
            writer.write(f"{key} = {value}\n")

        results.update(result)
        
#파일에 쓴 테스트 결과 확인
# !cat ~/aiffel/transformers/eval_results2.txt
!cat eval_results2.txt

## ****16-6. 프로젝트 : 커스텀 프로젝트 직접 만들기****

실습 코드에서 수행해 본 내용을 토대로, 이번에는 GLUE dataset의 `mnli` task를 수행하는 프로젝트를 커스텀 프로젝트 형태로 진행해 봅시다.  

`mnli` task는 이전 스텝에서 사용한 BERT를 사용하면 학습이 제대로 되지 않습니다. [여기](https://huggingface.co/models)를 참조하여 BERT가 아닌 다른 모델을 선택하세요.   tensorflow와 해당 모델에 대한 task로 검색하면 사용할 수 있는 모델이 나옵니다. 그 후 선택한 모델의 _tokenizer_와 해당 모델에 대한 *task* 와 *모델* 의 정보를 [여기](https://huggingface.co/transformers/index.html)에서 찾아 여러분의 프로젝트를 완성해 보세요.  

그냥 `run_glue.py`를 돌려보는 방식으로 진행하는 것을 원하는 것은 아닙니다. 아래와 같은 순서를 지켜서 진행해 주세요.  

****라이브러리 버전을 확인해 봅니다.****

In [3]:
import tensorflow
import numpy #LMS 같은 경우 1.21.4를 사용하고 있음
import pandas
import matplotlib
import json
import re

print(tensorflow.__version__)
print(numpy.__version__)
print(pandas.__version__)
print(matplotlib.__version__)
print(json.__version__)
print(re.__version__)

2.6.0
1.21.4
1.3.3
3.4.3
2.0.9
2.2.1


In [7]:
# !pip install transformers

In [4]:
import os
import numpy as np
from argparse import ArgumentParser
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures


### STEP 1. `mnli` 데이터셋을 분석해 보기

MNLI (Multi-Genre Natural Language Inference)  
https://cims.nyu.edu/~sbowman/multinli/

MNLI : 두 문장의 관계 판단(entailment, contradiction, neutral) 

https://huggingface.co/datasets/glue/viewer/mnli/train  
lavel  
0  :  entailment    : 함의  
1  :  neutral       : 중립  
2  :  contradiction : 모순  

함의(Entailment), 모순(Contradiction), 중립. (Neutral)

dataset 분할
'test_matched'	9,796  
'test_mismatched'	9,847  
'train'	392,702  
'validation_matched'	9,815  
'validation_mismatched'	9,832  

premise

hypothesis

text_a='The Clinton surrogates also held the high ground in the context war.'
text_b='The Clinton followers kept to the higher ground in the discussion. ', 
label='entailment'

text_a="We'd appreciate it.", 
text_b="We wouldn't like it very much.", 
label='contradiction'

text_a='Not on muscle.', 
text_b='On bone', 
label='contradiction'

text_a="It's been a pleasure discussing these issues with you.", 
text_b='I have liked talked with you.', 
label='entailment'

text_a='we were switching from one station to another and in between keeping the radio on', 
text_b="The radio was off, we didn't touch it.", 
label='contradiction'

text_a="You're always here for me.", 
text_b="You're never here when I need you.", 
label='contradiction'

text_a='The attempt was futile.', 
text_b='The attempt at fixing the road was pathetic. 
label='neutral'

text_a='Half-way through, Mr. Carter interrupted him to give a few cryptic orders through the telephone.', 
text_b='He was annoyed that Mr. Carter had corrupted him.'
label='neutral'

text_a="Today China's economy is more vulnerable to coordinated sanctions that would cut off its exports to industrial countries than ever before, yet it seems less open to democracy than ever before."
text_b="There are no reasons as to why China's economy is any more vulnerable."
label='contradiction'

text_a='Fitz Park itself is a fine example of the philanthropic attitude of the Victorians, who created green spaces in almost every town for the people to enjoy.', 
text_b='The Victorians removed all green spaces from almost every town.',
label='contradiction'

text_a='The good news, for those who must spend up to \$22,000 in a few days, is that there is a fairly broad latitude in what you can spend the money on.', 
text_b='There is nothing good for people who spend up to \$22,000', 
label='contradiction'




premise
전제

hypothesis
가설

text_a='클린턴 대리인도 상황전에서 우위를 점했습니다.'
text_b='클린턴 추종자들은 토론에서 더 높은 지위를 지켰습니다. ',
레이블 = '관련'

text_a="감사합니다.",
text_b="별로 좋아하지 않을 것입니다.",
레이블 = '모순'

text_a='근육이 아닙니다.',
text_b='뼈에',
레이블 = '모순'

text_a="이러한 문제에 대해 논의하게 되어 기쁩니다.",
text_b='나는 당신과 이야기하는 것을 좋아했습니다.',
레이블 = '관련'

text_a='한 방송국에서 다른 방송국으로 그리고 그 사이에 라디오를 켜고 있었습니다',
text_b="라디오가 꺼져 있어서 만지지 않았습니다.",
레이블 = '모순'

text_a="당신은 항상 나를 위해 여기에 있습니다.",
text_b="내가 당신을 필요로 할 때 당신은 여기 있지 않습니다.",
레이블 = '모순'

text_a='시도는 무의미했습니다.',
text_b='도로를 수리하려는 시도는 한심했습니다.
레이블 = '중립'

text_a='중간에 Carter 씨가 전화를 통해 몇 가지 비밀스러운 명령을 내리기 위해 그를 방해했습니다.',
text_b='그는 카터 씨가 그를 타락시켰다는 사실에 화가 났습니다.'
레이블 = '중립'

text_a="오늘날 중국 경제는 그 어느 때보다 공업국에 대한 수출을 차단하는 조정된 제재에 더 취약하지만 민주주의에 대해서는 그 어느 때보다 덜 개방적인 것 같습니다."
text_b="중국 경제가 더 취약한 이유는 없습니다."
레이블 = '모순'

text_a='Fitz Park 자체는 사람들이 즐길 수 있도록 거의 모든 도시에 녹지 공간을 만든 빅토리아 시대의 자선적 태도를 보여주는 좋은 예입니다.',
text_b='빅토리아인들은 거의 모든 도시에서 모든 녹지를 제거했습니다.',
레이블 = '모순'

text_a='며칠 안에 최대 $22,000를 지출해야 하는 사람들에게 희소식은 돈을 쓸 수 있는 범위가 상당히 넓다는 것입니다.',
text_b='22,000달러까지 쓰는 사람에게 좋은 것은 없습니다',
레이블 = '모순'

`tensorflow-datasets`를 이용하여 `glue/mnli`를 다운로드하려면 `tensorflow-datasets` 라이브러리 버전을 올려야 합니다.

In [6]:
# !pip install tensorflow-datasets -U

In [8]:
data, info = tfds.load('glue/mnli', with_info=True)
info.splits['train'].num_examples

INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: glue/mnli/2.0.0
INFO:absl:Load dataset info from /tmp/tmp96apwis4tfds
INFO:absl:Generating dataset glue (/aiffel/tensorflow_datasets/glue/mnli/2.0.0)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

INFO:absl:Downloading https://dl.fbaipublicfiles.com/glue/data/MNLI.zip into /aiffel/tensorflow_datasets/downloads/dl.fbaipublicfiles.com_glue_MNLIdNe8cK2kTBCG0bqBz2JxwShRT2KfuO3NVIwROTnjtfI.zip.tmp.e6e007f1d92e41ff9a1015a2db04cfb6...


...

Generating test_mismatched examples...:   0%|          | 0/9847 [00:00<?, ? examples/s]

Shuffling glue-test_mismatched.tfrecord...:   0%|          | 0/9847 [00:00<?, ? examples/s]

INFO:absl:Done writing glue-test_mismatched.tfrecord. Number of examples: 9847 (shards: [9847])
INFO:absl:Constructing tf.data.Dataset glue for split None, from /aiffel/tensorflow_datasets/glue/mnli/2.0.0


Dataset glue downloaded and prepared to /aiffel/tensorflow_datasets/glue/mnli/2.0.0. Subsequent calls will reuse this data.


392702

In [9]:
data['train'].take(1)

<TakeDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [10]:
examples = data['train'].take(1)
for example in examples:
    sentence1 = example['premise']
    sentence2 = example['hypothesis']
    label = example['label']
    print(sentence1)
    print(sentence2)
    print(label)

tf.Tensor(b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', shape=(), dtype=string)
tf.Tensor(b'Meaningful partnerships with stakeholders is crucial.', shape=(), dtype=string)
tf.Tensor(1, shape=(), dtype=int64)


In [11]:
examples

<TakeDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

### STEP 2. MNLIProcessor클래스 구현하기

In [14]:
class MnliProcessor(DataProcessor):
    """Processor for the MultiNLI data set (GLUE version)."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
#        warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
#         return ["0", "1"]
#         return ["0", "1", "2"]
        return ["entailment", "neutral", "contradiction"]
# 함의(Entailment), 모순(Contradiction), 중립. (Neutral)

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            text_b = line[4]
            label = None if set_type == "test" else line[0]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples 

mnli 이 데이터 셋은 문제가 있다.

transformers 에서 불러들이는 것과     ['contradiction', 'entailment', 'neutral']  
datasets load_dataset("glue", 'mnli') ['entailment', 'neutral', 'contradiction']  
방법으로 불러들이는 것은 라벨링이 다르게 되어있다.
(transformers가 틀렸다)  
https://github.com/huggingface/transformers/issues/12822    



test_matched, test_mismatched 데이터는   
라벨링이 전부 -1(load_dataset("glue", 'mnli'))로 되어있거나  
2(tensorflow_datasets)로 되어있다.  


In [15]:
import transformers
processor = transformers.glue_processors['mnli']()
label_list = processor.get_labels()
print(label_list)

['contradiction', 'entailment', 'neutral']


/opt/conda/lib/python3.9/site-packages/transformers/data/processors/glue.py:222: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)


In [16]:
import datasets
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("glue", 'mnli')
print(raw_datasets['validation_matched'].features['label'].names)

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /aiffel/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

['entailment', 'neutral', 'contradiction']


In [41]:
print(raw_datasets.keys())
print('label names :', raw_datasets['validation_matched'].features['label'].names)
test_dataset = raw_datasets['test_matched']
dit_test_dataset = test_dataset.to_dict()
print('label values :')
for i in range(10):
    print(dit_test_dataset['label'][i], end=' ')   

dict_keys(['train', 'validation_matched', 'validation_mismatched', 'test_matched', 'test_mismatched'])
label names : ['entailment', 'neutral', 'contradiction']
label values :
-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 

### glue 데이터셋 몇개를 꺼내 test data label이 제대로 되어있는지 확인

In [17]:
import transformers
processor = transformers.glue_processors['qqp']()
label_list = processor.get_labels()
print(label_list)

['0', '1']


C:\Users\ami\anaconda3\envs\python38\lib\site-packages\transformers\data\processors\glue.py:422: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)


In [18]:
import datasets
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("glue", 'qqp')


Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to C:\Users\ami\.cache\huggingface\datasets\glue\qqp\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [33]:
print(raw_datasets.keys())
print('label names :', raw_datasets['test'].features['label'].names)
test_dataset = raw_datasets['test']
dit_test_dataset = test_dataset.to_dict()
print('label values :')
for i in range(10):
    print(dit_test_dataset['label'][i], end=' ')   

dict_keys(['train', 'validation', 'test'])
label names : ['not_duplicate', 'duplicate']
label values :
-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 

In [42]:
import transformers
processor = transformers.glue_processors['mrpc']()
label_list = processor.get_labels()
print(label_list)

['0', '1']


In [34]:
import datasets
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("glue", 'mrpc')

Reusing dataset glue (C:\Users\ami\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [35]:
print(raw_datasets.keys())
print('label names :', raw_datasets['test'].features['label'].names)
test_dataset = raw_datasets['test']
dit_test_dataset = test_dataset.to_dict()
print('label values :')
for i in range(10):
    print(dit_test_dataset['label'][i], end=' ') 

dict_keys(['train', 'validation', 'test'])
label names : ['not_equivalent', 'equivalent']
label values :
1 1 1 0 0 1 0 1 1 0 

In [37]:
import datasets
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("glue", 'qqp')

Reusing dataset glue (C:\Users\ami\.cache\huggingface\datasets\glue\qqp\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [38]:
print(raw_datasets.keys())
print('label names :', raw_datasets['test'].features['label'].names)
test_dataset = raw_datasets['test']
dit_test_dataset = test_dataset.to_dict()
print('label values :')
for i in range(10):
    print(dit_test_dataset['label'][i], end=' ') 

dict_keys(['train', 'validation', 'test'])
label names : ['not_duplicate', 'duplicate']
label values :
-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 

#### 어느 것은 제대로 되어있고 어느 것은 분류를 하지 않은 것 같다.

In [17]:
processor = MnliProcessor()
examples = data['train'].take(1)

for example in examples:
    print('------원본데이터------')
    print(example)  
    example = processor.get_example_from_tensor_dict(example)
    print('------processor 가공데이터------')
    print(example)

------원본데이터------
{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Meaningful partnerships with stakeholders is crucial.'>, 'idx': <tf.Tensor: shape=(), dtype=int32, numpy=16399>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'>}
------processor 가공데이터------
InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeh

In [18]:
examples = (data['train'].take(1))
for example in examples:
    example = processor.get_example_from_tensor_dict(example)
    example = processor.tfds_map(example)
    print(example)

#InputExample(guid=1680, text_a='The identical rovers will act as robotic geologists , searching for evidence of past water .', text_b='The rovers act as robotic geologists , moving on six wheels .', label='0')

InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeholders is crucial.', label='neutral')


In [19]:
examples = (data['validation_matched'].take(30))
for example in examples:
    example = processor.get_example_from_tensor_dict(example)
    example = processor.tfds_map(example)
    print(example)

InputExample(guid=6287, text_a='uh-huh oh yeah all the people for right uh life or something', text_b='yeah lots of people for the right life ', label='entailment')
InputExample(guid=1579, text_a='Also, I will be assuming that the 6.0a cost of the Postal Service to take the mail from basic to workshared condition is constant as limited quantities of mail move back and forth between basic and workshared.', text_b=' I will be assuming that the 6.0a cost of the Postal Service to take the mail from basic to workshared condition is constant.', label='entailment')
InputExample(guid=9787, text_a="taken up by the oh okay oh so you know well that's i had wondered sometimes i knew that there was a lot of a lot of effort and a lot of work went into a lot of that and i just wondered if if it lasted and if it took you know like yeah", text_b='It cost a lot of money to put forth all that effort.  ', label='neutral')
InputExample(guid=7631, text_a='yeah i know the motor oil', text_b='I know nothing a

In [20]:
examples = (data['test_matched'].take(30))
for example in examples:
    example = processor.get_example_from_tensor_dict(example)
    example = processor.tfds_map(example)
    print(example)

InputExample(guid=5398, text_a='well is there a little electric pump you put in there', text_b='Is there a pump that you put in there?', label='contradiction')
InputExample(guid=1921, text_a="The Marquis de Sade's head is known to be above ground, as they say in the business.", text_b="The Marquis de Sade's head is safely kept underground.", label='contradiction')
InputExample(guid=3336, text_a='The Galilee Experience at the marina provides a 35-minute audio-visual introduction to the region; pleasure cruisers run to Capernaeum and other places; and you can also hire small craft.', text_b='Most visitors watch the video before they set out to explore the area.', label='contradiction')
InputExample(guid=2611, text_a="He'd better ask Dorcas, or one of the maids, if he wants to know about coffee-cups. ", text_b='Dorcas and the maids probably know about coffee cups.', label='contradiction')
InputExample(guid=1922, text_a='Burton, on the ropes after a particularly tough Tony Snow ( Fox News 

In [21]:
data['train']

<PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [22]:
type(examples)

tensorflow.python.data.ops.dataset_ops.TakeDataset

In [23]:
label_list = processor.get_labels()
label_list

['entailment', 'neutral', 'contradiction']

In [24]:
label_map = {label: i for i, label in enumerate(label_list)}
label_map

{'entailment': 0, 'neutral': 1, 'contradiction': 2}

In [25]:
num_classes = len(processor.get_labels())

### STEP 3. 위에서 구현한 processor 및 Huggingface에서 제공하는 tokenizer를 활용하여 데이터셋 구성하기

Natural Language Inference on MultiNLI  모델선택  
https://paperswithcode.com/sota/natural-language-inference-on-multinli

https://huggingface.co/docs/transformers/index

In [ ]:
# !pip install sentencepiece

In [26]:
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
model = TFAlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=num_classes)

Downloading:   0%|          | 0.00/60.1M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFAlbertForSequenceClassification.

Some layers of TFAlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
# from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=num_classes)
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-large-uncased')
# model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-large-uncased',num_labels=num_classes)

In [43]:
# from transformers import BertTokenizer, TFBertForSequenceClassification, AutoConfig
# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# model = TFBertForSequenceClassification.from_pretrained('bert-base-cased',num_labels=num_classes)

In [ ]:
# from transformers import XLNetTokenizer, TFXLNetForSequenceClassification
# tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
# model = TFXLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=num_classes)

tf_glue_convert_examples_to_features() 함수가 최종적으로 모델에 전달될 tf.data.Dataset 인스턴스를 생성

In [27]:
MAX_LENGTH = 128
BATCH_SIZE = 24

In [28]:
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=MAX_LENGTH, processor=processor)
train_dataset_batch = train_dataset.shuffle(100).batch(BATCH_SIZE).repeat(2)

Using label list ['entailment', 'neutral', 'contradiction']


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

...

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

*** Example ***
guid: 16399
features: InputFeatures(input_ids=[2, 19, 3514, 16, 158, 14978, 15, 644, 3862, 63, 577, 29153, 102, 179, 1485, 20, 11266, 14, 11870, 16, 14, 146, 2334, 5153, 17, 20, 4088, 16912, 16169, 29, 24757, 5414, 35, 4718, 68, 21, 78, 13, 7261, 2161, 8076, 128, 14, 1058, 11747, 17, 18560, 16, 1517, 687, 3234, 9, 3, 16912, 16169, 29, 24757, 25, 10421, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0

In [29]:
# validation 데이터셋
validation_dataset = tf_glue_convert_examples_to_features(data['validation_matched'], tokenizer, max_length=MAX_LENGTH, processor=processor)
validation_dataset_batch = validation_dataset.shuffle(100).batch(BATCH_SIZE)

Using label list ['entailment', 'neutral', 'contradiction']


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

*** Example ***
guid: 6287
features: InputFeatures(input_ids=[2, 10863, 8, 24600, 1962, 3979, 65, 14, 148, 26, 193, 10863, 201, 54, 301, 3, 3979, 7503, 16, 148, 26, 14, 193, 201, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0

In [30]:
# test 데이터셋
test_dataset = tf_glue_convert_examples_to_features(data['test_matched'], tokenizer, max_length=MAX_LENGTH, processor=processor)
test_dataset_batch = test_dataset.shuffle(100).batch(BATCH_SIZE)

Using label list ['entailment', 'neutral', 'contradiction']


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

*** Example ***
guid: 5398
features: InputFeatures(input_ids=[2, 134, 25, 80, 21, 265, 2098, 9786, 42, 442, 19, 80, 3, 25, 80, 21, 9786, 30, 42, 442, 19, 80, 60, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [31]:
examples = train_dataset.take(1)
for example in examples:
    print(example)

({'input_ids': <tf.Tensor: shape=(128,), dtype=int32, numpy=
array([    2,    19,  3514,    16,   158, 14978,    15,   644,  3862,
          63,   577, 29153,   102,   179,  1485,    20, 11266,    14,
       11870,    16,    14,   146,  2334,  5153,    17,    20,  4088,
       16912, 16169,    29, 24757,  5414,    35,  4718,    68,    21,
          78,    13,  7261,  2161,  8076,   128,    14,  1058, 11747,
          17, 18560,    16,  1517,   687,  3234,     9,     3, 16912,
       16169,    29, 24757,    25, 10421,     9,     3,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,  

### STEP 4. model을 생성하여 학습 및 테스트를 진행해 보기

In [32]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
model.summary()

Model: "tf_albert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
albert (TFAlbertMainLayer)   multiple                  11683584  
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 11,685,891
Trainable params: 11,685,891
Non-trainable params: 0
_________________________________________________________________


In [34]:
# 이전 스텝에서 배치처리를 진행한 데이터셋(xxxx_dataset_batch)을 활용
model.fit(train_dataset_batch, epochs=30, steps_per_epoch=115, 
                validation_data=validation_dataset_batch)

Epoch 1/30
115/115 [==============================] - 177s 1s/step - loss: 0.9492 - acc: 0.5351 - val_loss: 0.7675 - val_acc: 0.6710
Epoch 2/30
115/115 [==============================] - 161s 1s/step - loss: 0.7413 - acc: 0.6866 - val_loss: 0.7280 - val_acc: 0.6856
Epoch 3/30
115/115 [==============================] - 161s 1s/step - loss: 0.7252 - acc: 0.6964 - val_loss: 0.6935 - val_acc: 0.7043
Epoch 4/30
115/115 [==============================] - 161s 1s/step - loss: 0.7159 - acc: 0.6989 - val_loss: 0.6860 - val_acc: 0.7128
Epoch 5/30
115/115 [==============================] - 160s 1s/step - loss: 0.6702 - acc: 0.7221 - val_loss: 0.6459 - val_acc: 0.7407
Epoch 6/30
115/115 [==============================] - 160s 1s/step - loss: 0.6243 - acc: 0.7482 - val_loss: 0.6286 - val_acc: 0.7375
Epoch 7/30
115/115 [==============================] - 160s 1s/step - loss: 0.6147 - acc: 0.7536 - val_loss: 0.6159 - val_acc: 0.7483
Epoch 8/30
115/115 [==============================] - 160s 1s/step - 

In [35]:
result = model.evaluate(test_dataset_batch)
print(result)

409/409 [==============================] - 94s 229ms/step - loss: 1.9545 - acc: 0.3391
[1.9544824361801147, 0.33911800384521484]


In [36]:
import os
# output_dir = os.getenv('HOME')+'/aiffel/Huggingface'
# output_eval_file = os.path.join(output_dir, "eval_results.txt")
output_eval_file = "eval_results.txt"

with open(output_eval_file, "w") as writer:
    for i, v in enumerate(result) :
        if i == 0 :
            writer.write("Loss = %f\t" %(v))
        if i == 1 :
            writer.write("Accuracy = %f\n" %(v))
print("완료=3")

#파일에 쓴 테스트 결과 확인
# !cat ~/aiffel/Huggingface/eval_results.txt
!cat eval_results.txt

완료=3
Loss = 1.954482	Accuracy = 0.339118


****TFTrainer를 활용한 학습****

Huggingface의 `TFTrainer`를 활용해 학습을 진행

`TFTrainer`를 활용하기 위해서는 `TFTrainingArguments`에 학습 관련 설정을 미리 지정해 두어야 함.

In [37]:
# TFTrainer을 활용하는 형태로 모델 재생성
from transformers import (
    AutoConfig,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    PreTrainedTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
    glue_compute_metrics,
    glue_convert_examples_to_features,
    glue_output_modes,
    glue_processors,
    glue_tasks_num_labels,
)

# output_dir = os.getenv('HOME')+'/aiffel/Huggingface'
output_dir = '.'

training_args = TFTrainingArguments(
    output_dir=output_dir,           # output이 저장될 경로
    num_train_epochs=1,              # train 시킬 총 epochs
    per_device_train_batch_size=24,  # 각 device 당 batch size
    per_device_eval_batch_size=24,   # evaluation 시에 batch size
    warmup_steps=500,                # learning rate scheduler에 따른 warmup_step 설정
    weight_decay=0.01,               # weight decay
    logging_dir='./logs',            # log가 저장될 경로
    do_train=True,
    do_eval=True,
    eval_steps=115
)

max_seq_length=MAX_LENGTH
task_name = "mnli"

with training_args.strategy.scope():
#     model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=num_classes)    
    # model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)
#     model = TFXLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=num_classes)
#     model = TFBertForSequenceClassification.from_pretrained('bert-base-cased',num_labels=num_classes)
    model = TFAlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=num_classes)

All model checkpoint layers were used when initializing TFAlbertForSequenceClassification.

Some layers of TFAlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


아래에서 생성하게 될 TFTrainer의 인자로 넘겨주어야 할 것 중에 compute_metrics 메소드가 있습니다. 이것은 task가 classification인지 regression인지에 따라 모델의 출력 형태가 달라지므로 task별로 적합한 출력 형식을 고려해 모델의 성능을 계산하는 방법을 미리 지정

In [38]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    if output_mode == "classification":
        preds = np.argmax(p.predictions, axis=1)
    elif output_mode == "regression":
        preds = np.squeeze(p.predictions)
    return glue_compute_metrics(task_name, preds, p.label_ids)

output_mode = glue_output_modes[task_name]
output_mode

'classification'

TFTrainer를 활용할 때 데이터셋에서 잊지 않아야 할 것이 tf.data.experimental.assert_cardinality()를 데이터셋에 적용해 주는 것입니다. 이를 호출해 주지 않으면 TFTrainer.train()에서 assert fail이 발생

In [39]:
train_dataset      = train_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['train'].num_examples))
validation_dataset = validation_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['validation_matched'].num_examples))
test_dataset       = test_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['test_matched'].num_examples))

In [40]:
trainer = TFTrainer(
    model=model,                       # 학습시킬 model
    args=training_args,                # TFTrainingArguments을 통해 설정한 arguments
    train_dataset=train_dataset,       # training dataset
    eval_dataset=validation_dataset,   # evaluation dataset
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.9/site-packages/transformers/trainer_tf.py:109: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/master/examples/tensorflow
  warnings.warn(


In [42]:
Training
if training_args.do_train:
    trainer.train()
    trainer.save_model()
    tokenizer.save_pretrained(training_args.output_dir)

In [ ]:
import numpy as np

# Evaluation
results = {}
if training_args.do_eval:
    result = trainer.evaluate()
    output_eval_file = os.path.join(training_args.output_dir, "eval_results2.txt")

    with open(output_eval_file, "w") as writer:
        for key, value in result.items():
            writer.write(f"{key} = {value}\n")

        results.update(result)
        
#파일에 쓴 테스트 결과 확인
!cat eval_results2.txt

**힌트**

혹시 STEP 2의 진행에 어려움을 겪고 계신다면 transformer 프로젝트 내부를 살펴보시면 참고할만한 예시 코드를 찾아볼 수 있을 것입니다. transformers의 공식 [github](https://github.com/huggingface/transformers) 을 참고하는 것도 좋은 방법이에요!

## 후기:  

### 교육된모델 선택  
처음엔 모델명 뒤에 붙은게 정확히 무엇을 뜻하는 것인지 몰라서 이것저것 가져다 붙여보고 에러뜨고 헤멨다.  
ForSequenceClassification, ForMultipleChoice, ForTokenClassification, ForQuestionAnswering 이런 것들 말이다.  

모델선택하는데 아래와 같은 사이트를 참고했다.  
Natural Language Inference on MultiNLI   
https://paperswithcode.com/sota/natural-language-inference-on-multinli  

https://huggingface.co/docs/transformers/index  




### Glue mnli 데이터셋 라벨명칭 문제
STEP 2 MNLIProcessor클래스 구현하기 에서는 받아진 데이터 내용과 인터넷에서 구한 소스 라벨링이 다르게 되어있는 것을 확인했다.  
데이터를 확인해서 내용에 맞게 라벨명을 고쳐넣었다.(프로그램은 라벨명 신경쓰지 않고 그냥 돌리면 된다)  


나중에 확인해보니 Glue mnli 데이터 셋은 라벨링 문제가 있었다.  

transformers 에서 불러들이는 것과 ['contradiction', 'entailment', 'neutral']  
datasets load_dataset("glue", 'mnli') ['entailment', 'neutral', 'contradiction']  
방법으로 불러들이는 것은 라벨링이 다르게 되어있다. (transformers가 틀렸다)  
https://github.com/huggingface/transformers/issues/12822  


### 모델 지정
그다음엔 모델을 지정하는데  num_labels을 넣어주지 않아서 시간낭비했다.  
model = TFAlbertForSequenceClassification.from_pretrained("albert-base-v2", ***num_labels=num_classes***)



### 테스트셋 라벨  문제

프로그램 돌리고 테스트셋을 돌린 결과를 확인해 보니 학습이 안된 것 처럼 보였다.  

Epoch 30/30 loss: 0.5550 - acc: 0.7761 - val_loss: 0.5093 - val_acc: 0.8041  
result = model.evaluate(test_dataset_batch)  
[1.95, 0.33]  

val_acc: 0.8041로 나왔으니 테스트셋에 문제가 있는 것 같아서 확인해 보니 라벨값이 전부 2로 되어있다.  

test_matched, test_mismatched 데이터는  
라벨링이 전부 -1(load_dataset("glue", 'mnli'))로 되어있거나  
2(tensorflow_datasets)로 되어있다.  

glue 데이터셋 몇개의 테스트셋을 확인해 보니 어떤 것은 라벨링이 제대로 되어있고
어떤 것은 -1 이런 식으로 라벨을 달아놓지 않았다.  
무슨 목적이 있어서 그래놓았겠지.


### TFTrainer를 활용한 학습 문제  
1 epoch를 걸어놓았는데도 2시간이 넘도록 결과가 나오지 않는다. 무언가 잘못 설정한 것 같다.  
아마도 인코딩 과정에서 max_length를 넘어가는 데이터는 삭제가 되는 것 같은데  
tf.data.experimental.assert_cardinality()를 삭제되기 전 데이터 수로 넣어서 그런 것 같다.  

 FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers.  
 We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object.  
 
 TFTrainer는 다음 버전에서 없어진다니까 그냥 포기  